In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import cv2
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 0)

from nos.client import InferenceClient, TaskType

### Inference Client API

In [ ]:
client = InferenceClient()

In [ ]:
client.WaitForServer()

In [ ]:
client.IsHealthy()

### List all available models

In [ ]:
models = client.ListModels()
models_df = pd.DataFrame(models)
models_df

### Get model input/output signatures

In [ ]:
signatures = [client.GetModelInfo(spec).signature for spec in models]
pd.DataFrame(signatures)

### Load image

In [ ]:
import requests
from PIL import Image

url = "https://raw.githubusercontent.com/open-mmlab/mmdetection/main/demo/demo.jpg"
img = Image.open(requests.get(url, stream=True).raw).resize((640, 480))
images = [img]
img

### Run object detection

In [ ]:
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/small", images=images)
for idx, (bboxes, scores, labels) in enumerate(zip(predictions["bboxes"], predictions["scores"], predictions["scores"])):
    print(idx, bboxes.shape, scores.shape, labels.shape)

In [ ]:
%%timeit -n 10 -r 10
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/small", images=images)

In [ ]:
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/medium", images=images)

In [ ]:
%%timeit -n 100
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/medium", images=images)

### Visualize object detection

In [ ]:
def visualize_det2d(vis: np.ndarray, bboxes: np.ndarray, labels: np.ndarray) -> np.ndarray:
    for bbox, label in zip(bboxes.astype(np.int32), labels):
        cv2.rectangle(vis, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    return vis

In [ ]:
for idx, (img, bboxes, scores, labels) in enumerate(zip(images, predictions["bboxes"], predictions["scores"], predictions["scores"])):
    display(Image.fromarray(visualize_det2d(np.asarray(img).copy(), bboxes, labels)))

In [ ]:
# Filter models by task
predictions = {}
for model_name in models_df[models_df.task == TaskType.OBJECT_DETECTION_2D].name:
    print(model_name)
    predictions[model_name] = client.Run(TaskType.OBJECT_DETECTION_2D, model_name, images=images)

In [ ]:
predictions

In [ ]:
predictions_vis = []
for name, pred in predictions.items():
    if pred is None: 
        continue
    for idx, (img, bboxes, scores, labels) in enumerate(zip(images, pred["bboxes"], pred["scores"], pred["labels"])):
        print(name)
        vis = Image.fromarray(visualize_det2d(np.asarray(img), bboxes, labels))
        predictions_vis.append(vis)

In [ ]:
import math
from PIL import Image, ImageDraw, ImageFont


def image_grid(imgs, rows, cols, captions = None):
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    if captions is None:
        captions = [f"image_{i}" for i in range(len(imgs))]
    for i, (img, caption) in enumerate(zip(imgs, captions)):
        draw = ImageDraw.Draw(img)
        draw.text((0, 0), caption, (255,255,255))
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


display(image_grid(predictions_vis, math.ceil(len(predictions_vis) / 3.), 3, captions=list(predictions.keys())))

### Run object detection on a full video

In [ ]:
det2d = client.Module(TaskType.OBJECT_DETECTION_2D, "yolox/medium-trt")
predictions = det2d(images=images)

In [ ]:
%%timeit -n 100
predictions = det2d(images=images)

In [ ]:
import uuid
from itertools import islice
from tqdm import tqdm
from PIL import Image
import numpy as np

from nos.common.io import VideoReader, VideoWriter

video = VideoReader("/data/videos/tests/video_intelligence/test_video_intelligence.mp4")

filename = f"{str(uuid.uuid4())}.mp4"
writer = VideoWriter(filename)
for idx in tqdm(range(0, len(video), 10)):
    img = Image.fromarray(video[idx])
    W, H = img.size
    img = img.resize((640, 480))
    pred = det2d(images=[img])
    for (bboxes, scores, labels) in zip(pred["bboxes"], pred["scores"], pred["labels"]):
        vis = visualize_det2d(np.asarray(img).copy(), bboxes, labels)
    vis = cv2.resize(vis, (W, H), interpolation=cv2.INTER_AREA)
    writer.write(vis)
writer.close()
print(f"Written video: {filename}")

In [ ]:
from IPython.display import Video

Video(filename, width=W, height=H, embed=True)